# Leveraging Neo4j for Service Identification and Microservices Partitioning in Business Process Systems

# 1. Install Required Libraries

In [ ]:
!pip install neo4j torch

# 2. Import Libraries

Import all necessary libraries for XML parsing, Neo4j interaction, GPU detection, and concurrent processing.

In [40]:
# Import Libraries
import xml.etree.ElementTree as ET
from neo4j import GraphDatabase
import os
import re
import html
import torch 
from concurrent.futures import ThreadPoolExecutor

# 3. Check CUDA Availability

Detect whether CUDA (GPU) is available on your system. This information will be printed at the beginning of the notebook.

In [41]:
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

CUDA available: True
CUDA version: 11.8
Using device: cuda


In [42]:
# Function to check CUDA availability
def check_cuda():
    if torch.cuda.is_available():
        print("CUDA is available. GPU will be used if applicable.")
        print(f"Device Name: {torch.cuda.get_device_name(0)}")
    else:
        print("CUDA is not available. Using CPU.")

# Execute CUDA check
check_cuda()

CUDA is available. GPU will be used if applicable.
Device Name: NVIDIA GeForce RTX 3060 Laptop GPU


# 4. Define Connection to Neo4j

In [43]:
# Neo4j connection details
uri = "bolt://localhost:7687"
username = "neo4j"
password = "170202Kcf"

# Create a driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))

In [44]:
def test_connection():
    try:
        with driver.session(database="erpbpmn") as session:
            result = session.run("RETURN 1 AS test")
            for record in result:
                print(f"Connection successful, test query result: {record['test']}")
    except Exception as e:
        print(f"Failed to connect to Neo4j: {e}")

test_connection()

Connection successful, test query result: 1


# 5. Define Functions to Create Nodes and Relationships

In [45]:
# Define functions to create nodes and relationships
def create_node(tx, label, properties):
    color_map = {
        'Task': '#ADD8E6',
        'StartEvent': '#90EE90',
        'EndEvent': '#FFB6C1',
        'ExclusiveGateway': '#FFD700',
        'Element': '#FFA07A'
    }
    color = color_map.get(label, '#D3D3D3')

    query = (
        f"MERGE (n:{label} {{id: $properties.id}}) "
        "SET n += $properties, n.color = $color "
        "RETURN n"
    )
    result = tx.run(query, properties=properties, color=color)
    return result.single()[0]

def create_relationship_with_id(tx, label1, id1, label2, id2, rel_type, properties):
    rel_color_map = {
        'SEQUENCE_FLOW': '#A9A9A9',
    }
    color = rel_color_map.get(rel_type, '#696969')

    query = (
        f"MATCH (a:{label1} {{id: $id1}}), (b:{label2} {{id: $id2}}) "
        f"MERGE (a)-[r:{rel_type} {{id: $properties.id}}]->(b) "
        "SET r += $properties, r.color = $color "
        "RETURN r"
    )
    result = tx.run(query, id1=id1, id2=id2, properties=properties, color=color)
    record = result.single()
    if record:
        return record[0]
    else:
        print(f"Warning: Could not create relationship {rel_type} between {label1}({id1}) and {label2}({id2}). One of the nodes may not exist.")
        return None

# 6. Parse BPMN XML Files and Load into Neo4j

In [46]:
# Define functions to parse BPMN XML files
def clean_name(name):
    name = re.sub('<[^<]+?>', '', name)
    name = html.unescape(name)
    return name.strip()

In [47]:
# Function to parse draw.io BPMN XML files
def parse_drawio_bpmn_xml(file_path, level, module, activity=None):
    tree = ET.parse(file_path)
    root = tree.getroot()

    tasks = []
    events = []
    gateways = []
    sequence_flows = []
    
    namespace = {'mx': 'http://www.mxgraph.com/2006/mxGraphModel'}

    for cell in root.findall('.//mxCell'):
        style = cell.get('style', '').lower()
        cell_id = cell.get('id')
        value = clean_name(cell.get('value', ''))
        if 'shape=mxgraph.bpmn.task' in style:
            tasks.append({'id': f"{level}_{cell_id}", 'name': value, 'level': level, 'module': module, 'activity': activity})
        elif 'shape=mxgraph.bpmn.event' in style:
            if 'outline=end' in style:
                events.append({'id': f"{level}_{cell_id}", 'name': value, 'type': 'EndEvent', 'level': level, 'module': module, 'activity': activity})
            else:
                events.append({'id': f"{level}_{cell_id}", 'name': value, 'type': 'StartEvent', 'level': level, 'module': module, 'activity': activity})
        elif 'shape=mxgraph.bpmn.gateway' in style or 'shape=mxgraph.bpmn.gateway2' in style:
            gateways.append({'id': f"{level}_{cell_id}", 'name': value, 'type': 'ExclusiveGateway', 'level': level, 'module': module, 'activity': activity})
        elif cell.get('edge') == '1':
            sequence_flows.append({
                'id': f"{level}_{cell_id}",
                'sourceRef': f"{level}_{cell.get('source')}",
                'targetRef': f"{level}_{cell.get('target')}",
                'name': value,
                'level': level,
                'module': module,
                'activity': activity
            })

    return tasks, events, gateways, sequence_flows, root

In [48]:
# Helper function to determine the label of an element by its ID
def get_element_label_by_id(root, element_id):
    try:
        level, cell_id = element_id.split('_', 1)
    except ValueError:
        cell_id = element_id
    for cell in root.findall('.//mxCell'):
        if cell.get('id') == cell_id:
            style = cell.get('style', '').lower()
            if 'shape=mxgraph.bpmn.task' in style:
                return 'Task'
            elif 'shape=mxgraph.bpmn.event' in style:
                if 'outline=end' in style:
                    return 'EndEvent'
                else:
                    return 'StartEvent'
            elif 'shape=mxgraph.bpmn.gateway' in style or 'shape=mxgraph.bpmn.gateway2' in style:
                return 'ExclusiveGateway'
            else:
                return 'Element'
    return 'Element'

In [50]:
# Main function to process all BPMN files with parallel processing
def process_bpmn_file(session, filename, file_path, level, module, activity):
    print(f"\nProcessing {filename} at Level {level}...")
    tasks, events, gateways, sequence_flows, root = parse_drawio_bpmn_xml(file_path, level, module, activity)

    print(f"Parsed {len(tasks)} tasks, {len(events)} events, {len(gateways)} gateways, {len(sequence_flows)} sequence flows from {filename}")

    for task in tasks:
        session.execute_write(create_node, 'Task', task)
    for event in events:
        session.execute_write(create_node, event['type'], event)
    for gateway in gateways:
        session.execute_write(create_node, gateway['type'], gateway)

    for flow in sequence_flows:
        source_id = flow['sourceRef']
        target_id = flow['targetRef']
        rel_properties = {'id': flow['id'], 'name': flow.get('name'), 'level': level, 'module': module, 'activity': activity}
        source_label = get_element_label_by_id(root, source_id)
        target_label = get_element_label_by_id(root, target_id)
        rel_created = session.execute_write(
            create_relationship_with_id,
            source_label, source_id, target_label, target_id,
            'SEQUENCE_FLOW', rel_properties
        )
        if rel_created:
            print(f"Created SEQUENCE_FLOW relationship from {source_label}({source_id}) to {target_label}({target_id})")
        else:
            print(f"Failed to create SEQUENCE_FLOW relationship from {source_id} to {target_id}")

# 7. Main Execution

In [51]:
# Run the processing function with parallelization
if __name__ == "__main__":
    check_cuda()
    test_connection()
    bpmn_dir = './assets'
    filenames = [f for f in os.listdir(bpmn_dir) if f.endswith('.xml')]

    with driver.session(database="erpbpmn") as session:
        with ThreadPoolExecutor(max_workers=4) as executor:
            futures = []
            for filename in filenames:
                file_path = os.path.join(bpmn_dir, filename)
                match = re.match(r'BPMN\s+(.+)\s+Level\s+(\d)(?:\s+-\s+(.+))?\.xml', filename)
                if match:
                    module = match.group(1)
                    level = int(match.group(2))
                    activity = match.group(3) if match.group(3) else None
                    tree = ET.parse(file_path)
                    root = tree.getroot()
                    futures.append(executor.submit(process_bpmn_file, session, filename, file_path, level, module, activity))
                else:
                    print(f"Skipping file {filename} as it doesn't match the expected naming pattern.")
            
            for future in futures:
                try:
                    future.result()
                except Exception as e:
                    print(f"Error processing file: {e}")


CUDA is available. GPU will be used if applicable.
Device Name: NVIDIA GeForce RTX 3060 Laptop GPU
Connection successful, test query result: 1

Processing BPMN Account Payable Level 1.xml at Level 1...
Parsed 1 tasks, 2 events, 0 gateways, 2 sequence flows from BPMN Account Payable Level 1.xml

Processing BPMN Account Payable Level 2.xml at Level 2...
Parsed 21 tasks, 2 events, 11 gateways, 38 sequence flows from BPMN Account Payable Level 2.xml

Processing BPMN Account Payable Level 3 - Creating the Purchase DP Invoice.xml at Level 3...
Parsed 6 tasks, 2 events, 3 gateways, 17 sequence flows from BPMN Account Payable Level 3 - Creating the Purchase DP Invoice.xml

Processing BPMN Account Payable Level 3 - Finalizing Purchase DP Invoice Document.xml at Level 3...
Parsed 3 tasks, 2 events, 0 gateways, 8 sequence flows from BPMN Account Payable Level 3 - Finalizing Purchase DP Invoice Document.xml
Skipping file BPMN Level 0.xml as it doesn't match the expected naming pattern.

Processing

Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Transaction failed and will be retried in 1.142172462077642s (Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687))))
Transaction failed and will be retried in 0.9699964954995002s (Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687))))


Error processing file: Existing exports of data: object cannot be re-sized
Error processing file: Existing exports of data: object cannot be re-sized
Error processing file: Existing exports of data: object cannot be re-sized
Error processing file: Existing exports of data: object cannot be re-sized


Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))
Transaction failed and will be retried in 1.002232090523628s (Failed to read from defunct connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687))))


Created SEQUENCE_FLOW relationship from StartEvent(3_4) to Task(3_5)
Failed to create SEQUENCE_FLOW relationship from 3_9 to 3_5
Failed to create SEQUENCE_FLOW relationship from 3_8 to 3_9
Failed to create SEQUENCE_FLOW relationship from 3_12 to 3_13
Created SEQUENCE_FLOW relationship from Task(3_16) to ExclusiveGateway(3_19)
Created SEQUENCE_FLOW relationship from Task(3_5) to Task(3_16)
Created SEQUENCE_FLOW relationship from ExclusiveGateway(3_19) to ExclusiveGateway(3_20)
Created SEQUENCE_FLOW relationship from ExclusiveGateway(3_19) to Task(3_6)
Created SEQUENCE_FLOW relationship from ExclusiveGateway(3_20) to Task(3_23)
Failed to create SEQUENCE_FLOW relationship from 3_23 to 3_12
Created SEQUENCE_FLOW relationship from ExclusiveGateway(3_28) to EndEvent(3_14)
Created SEQUENCE_FLOW relationship from Task(3_6) to ExclusiveGateway(3_28)
Created SEQUENCE_FLOW relationship from Task(3_23) to ExclusiveGateway(3_28)
Error processing file: Expected structure, found marker 66


# 8. Verification query

In [52]:
def verify_data_import():
    with driver.session(database="erpbpmn") as session:
        result = session.run("MATCH (n) RETURN labels(n) AS Label, count(n) AS Count ORDER BY Count DESC")
        print("Node counts by label:")
        for record in result:
            print(f"{record['Label']}: {record['Count']}")
        
        result = session.run("MATCH ()-[r]->() RETURN type(r) AS RelationType, count(r) AS Count ORDER BY Count DESC")
        print("\nRelationship counts by type:")
        for record in result:
            print(f"{record['RelationType']}: {record['Count']}")

verify_data_import()

Node counts by label:
['Task']: 5
['ExclusiveGateway']: 3
['StartEvent']: 2
['EndEvent']: 2

Relationship counts by type:
SEQUENCE_FLOW: 10
